In [ ]:
import os
import pandas as pd
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Define the path to the results directory
results_dir = '../results/CWRU Load Division IMGNET Pretrain'

# Define the mapping from class indices to class names
class_mapping = {
    0: 'Class A',
    1: 'Class B',
    2: 'Class C',
    3: "Class D"
}

# Initialize lists to store all predictions and true labels
all_preds = []
all_true = []

# Iterate over each fold
fold_dirs = sorted([d for d in os.listdir(results_dir) if d.startswith('fold')])
for fold_dir in fold_dirs:
    fold_path = os.path.join(results_dir, fold_dir)
    preds_true_file = os.path.join(fold_path, 'preds_true.csv')
    
    # Read the predictions and true labels
    df = pd.read_csv(preds_true_file)
    preds = df['preds']
    true = df['true']
    
    # Append to the overall lists
    all_preds.extend(preds)
    all_true.extend(true)
    
    # Compute the confusion matrix for this fold
    #cm = confusion_matrix(true, preds)
    
    # Plot the confusion matrix for this fold
    #plt.figure(figsize=(8, 6))
    #sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_mapping.values(), 
                yticklabels=class_mapping.values())
    #plt.title(f'Confusion Matrix for {fold_dir}')
    #plt.xlabel('Predicted')
    #plt.ylabel('True')
    #plt.show()

# Compute the general confusion matrix across all folds
general_cm = confusion_matrix(all_true, all_preds)

# Plot the general confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(general_cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_mapping.values(), 
            yticklabels=class_mapping.values())
plt.title('General Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

results_dir = '../results'

# Define the metrics to analyze
metrics = ['f1_score', 'accuracy_score']

# Initialize a dictionary to store metrics for each method
method_metrics = {}

# Iterate over each method folder
method_folders = [d for d in os.listdir(results_dir) if os.path.isdir(os.path.join(results_dir, d))]
for method in method_folders:
    method_path = os.path.join(results_dir, method)
    
    # Initialize a list to store metrics for all folds of this method
    method_metrics[method] = {metric: [] for metric in metrics}
    
    # Iterate over each fold in the method folder
    fold_dirs = sorted([d for d in os.listdir(method_path) if d.startswith('fold')])
    for fold_dir in fold_dirs:
        fold_path = os.path.join(method_path, fold_dir)
        test_results_file = os.path.join(fold_path, 'test_results.csv')
        
        # Read the test results for this fold
        df = pd.read_csv(test_results_file)
        
        # Append metrics to the method's list
        for metric in metrics:
            method_metrics[method][metric].append(df[metric].values[0])

# Convert the metrics into a DataFrame for easier plotting
data = []
for method, metrics_dict in method_metrics.items():
    for metric, values in metrics_dict.items():
        for value in values:
            data.append({'Method': method, 'Metric': metric, 'Value': value})
df = pd.DataFrame(data)

# Bar Plot: Mean of each metric for each method (one chart at a time)
for metric in metrics:
    plt.figure(figsize=(8, 6))
    sns.barplot(data=df[df['Metric'] == metric], x='Method', y='Value', palette='viridis', hue='Method')
    plt.title(f'Mean {metric} with Standard Deviation')
    plt.xlabel('Method')
    plt.ylabel(metric)
    plt.tight_layout()
    plt.show()

# Box Plot: Distribution of each metric for each method (one chart at a time)
for metric in metrics:
    plt.figure(figsize=(8, 6))
    sns.boxplot(data=df[df['Metric'] == metric], x='Method', y='Value', palette='viridis', hue='Method')
    plt.title(f'Distribution of {metric}')
    plt.xlabel('Method')
    plt.ylabel(metric)
    plt.tight_layout()
    plt.ylim(0, 1.01)
    plt.show()

# Create a summary table for each method
summary_table = pd.DataFrame(columns=['Method', 'Metric', 'Mean', 'Std'])

for method in method_metrics:
    for metric in metrics:
        mean_value = pd.Series(method_metrics[method][metric]).mean()
        std_value = pd.Series(method_metrics[method][metric]).std()
        summary_table = summary_table.append({
            'Method': method,
            'Metric': metric,
            'Mean': mean_value,
            'Std': std_value
        }, ignore_index=True)

# Pivot the table for better readability
summary_table_pivot = summary_table.pivot(index='Method', columns='Metric', values=['Mean', 'Std'])

In [ ]:
summary_table_pivot